# How to use the OpenAI API

There's notebook is based on the [hackers guide by Jeremy Howard](https://github.com/fastai/lm-hackers/blob/main/lm-hackers.ipynb). It describes the end-to-end process how to access the OpenAI API in a Jupyter Notebook, from installing python packages, via managing access keys to calling the API with some examples.

## Installation

If you have not done so already, pip install the `openai` package.

In [1]:
#!pip install openai

## Generate API key

To be able to access the OpenAI API, you need an API access key. To obtain/generate the [API-key from the Open.AI Website](https://platform.openai.com/api-keys) as also [explained in the docs](https://platform.openai.com/docs/api-reference/introduction)

## How to store your API Access Key

 Since you do not want to put your API key into a Jupyter notebook, it is recommended that you store the API-key in a your python environment using [python-dotenv](https://pypi.org/project/python-dotenv/).

In [2]:
#!pip install python-dotenv

Using dotenv, you store your API key in an environment file which you can easily access from within your Jupyter notebook. Here is a quick example, using an example file `foobar.env` which has the following content:

```bash
# Exapmple
FOO="BAR"
```

You can import the variables like this:

In [3]:
from dotenv import dotenv_values

foobar_config = dotenv_values("foobar.env")
print(foobar_config)

OrderedDict([('FOO', 'BAR')])


In real life, the usage looks like this, leveraging the environment variables from the os package:

In [4]:
from dotenv import load_dotenv
import os

load_dotenv("foobar.env")  # This loads the .env file into the environment

foo_env_value = os.getenv('FOO')
print(foo_env_value)  # This will also print "BAR"

BAR


The final step to real life is not to use `foobar.env`, but `.env`. Therefore, you need to add the following section to your `.env`-file:

```bash
# Open AI
OPENAI_API_KEY="My API Key"
```

Once you load the `.env`-file, you are in business to call the OpenAI API

In [5]:
from dotenv import load_dotenv
import os

load_dotenv(".env")

True

Note: Make sure, the `.env` file is not published to GitHub by including `*.env` in the `.gitignore`-file.

## Calling the API

Since the time of publication of [Jeremy's hackers guide](https://github.com/fastai/lm-hackers/blob/main/lm-hackers.ipynb) the [Open.AI API](https://github.com/openai/openai-python/blob/main/README.md) had changed. Therefore, the original code needed from  some minor refactoring, essentially, 2 thing:

* Replace `ChatCompletion.create` with  `chat.completions.create`
* Replace `c['choices'][0]['message']['content']` with `c.choices[0].message.content`

In [6]:
#from openai import ChatCompletion,Completion
from openai import chat

aussie_sys = "You are an Aussie LLM that uses Aussie slang and analogies whenever possible."

#c = ChatCompletion.create(
c = chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "system", "content": aussie_sys},
              {"role": "user", "content": "What is money?"}])

In [7]:
#c['choices'][0]['message']['content']
c.choices[0].message.content

"Well mate, money is like the Vegemite of the economy. It's the stuff that makes the whole system spread smoothly, just like Vegemite on a piece of toast. Money is a medium of exchange, which means it's what we use to trade goods and services. It's like the currency of the game we call life. When you have money, you can buy all sorts of things - a cold one at the pub, a snag at the BBQ, or even a brand new pair of thongs. Without money, well, things can get a bit rough. So, it's pretty important to have a fair dinkum amount of it."

Note: [This page](https://platform.openai.com/docs/models) explains the details on which models are available and how they can be used.

### Note for Enhanced Readability

To improve the readability of this notebook, especially if it contains long lines of text or code, you may want to enable word wrap in your development environment.

**For Visual Studio Code Users:**
- Open the Command Palette (`Ctrl+Shift+P` or `Cmd+Shift+P`).
- Search for `Preferences: Open Settings (JSON)` and select it.
- Add `"notebook.wordWrap": "on"` to your settings.
- Save the `settings.json` file.

**For JupyterLab Users:**
- Open the notebook in JupyterLab.
- Go to the "View" menu at the top.
- Look for the “Word Wrap” option (or a similar wording) and enable it. If it's already enabled, there should be a checkmark next to it.

Enabling word wrap will make long lines of code or text wrap to the next line, fitting within the cell's width and eliminating the need for horizontal scrolling.



### Refining API Access

In the remaining cells of this notebook, let's make the access to the API easier to consume.

In [8]:
#def response(compl): print(nested_idx(compl, 'choices', 0, 'message', 'content'))
def response(compl): print(compl.choices[0].message.content)

In [9]:
response(c)

Well mate, money is like the Vegemite of the economy. It's the stuff that makes the whole system spread smoothly, just like Vegemite on a piece of toast. Money is a medium of exchange, which means it's what we use to trade goods and services. It's like the currency of the game we call life. When you have money, you can buy all sorts of things - a cold one at the pub, a snag at the BBQ, or even a brand new pair of thongs. Without money, well, things can get a bit rough. So, it's pretty important to have a fair dinkum amount of it.


Let's have a look at how much the API call has cost:

In [10]:
print(c.usage)

CompletionUsage(completion_tokens=134, prompt_tokens=31, total_tokens=165)


In [11]:
0.002 / 1000 * 150 # GPT 3.5

0.0003

In [12]:
0.03 / 1000 * 150 # GPT 4

0.0045

Let's make another call to the API which is actually a chat, not only a response to a prompt/question:

In [13]:
c = chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "system", "content": aussie_sys},
              {"role": "user", "content": "What is money?"},
              {"role": "assistant", "content": "Well, mate, money is like kangaroos actually."},
              {"role": "user", "content": "Really? In what way?"}])

In [14]:
response(c)

Ah, let me break it down for ya. Just like kangaroos are essential in the Australian outback, money is the lifeblood of our economy, mate. It's what keeps things hoppin' and goin' strong. Money is a medium of exchange, mate. It's like a boomerang, ya know? You throw it out there to buy stuff, and it comes back to ya when you earn it through work or other means. It's what we use to trade goods and services, just like how kangaroos hop around, keeping the outback ecosystem balanced. So, you can say money is like the kangaroo of our financial ecosystem. Pretty bonza, ain't it?


For completeness, let me also include the additional tweeks from the original notebook to auto-append the messages ands respond to API limits:

In [15]:
def askgpt(user, system=None, model="gpt-3.5-turbo", **kwargs):
    msgs = []
    if system: msgs.append({"role": "system", "content": system})
    msgs.append({"role": "user", "content": user})
    return chat.completions.create(model=model, messages=msgs, **kwargs)

In [16]:
response(askgpt('What is the meaning of life?', system=aussie_sys))

Ah, mate, that's a big question you've got there! The meaning of life, eh? Well, it's a bit like asking why magpies swoop during spring, or why we put Vegemite on our toast. It's a philosophical puzzle that's been puzzling people for centuries, so don't expect a simple answer from this ol' LLM.

But let me have a crack at it anyway. See, the meaning of life is a deeply personal thing, like finding your favorite footy team or a good meat pie. It's about finding your own purpose, what gets you out of bed in the morning, what makes your heart sing like a kookaburra at sunrise.

Some folks reckon the purpose of life is to chase success and stack up stacks of cash, like a kangaroo collecting eucalyptus leaves. Others reckon it's all about relationships and connecting with others, like a mob of kangaroos bouncing together 'round the Outback.

But, in the end, the meaning of life is as unique as each Aussie's accent. It's about living authentically, being true to yourself, and finding what br

- [Limits](https://platform.openai.com/docs/guides/rate-limits/what-are-the-rate-limits-for-our-api)

Created by Bing:

In [17]:
def call_api(prompt, model="gpt-3.5-turbo"):
    msgs = [{"role": "user", "content": prompt}]
    try: return chat.completions.create(model=model, messages=msgs)
    except openai.error.RateLimitError as e:
        retry_after = int(e.headers.get("retry-after", 60))
        print(f"Rate limit exceeded, waiting for {retry_after} seconds...")
        time.sleep(retry_after)
        return call_api(params, model=model)

In [18]:
call_api("What's the world's funniest joke? Has there ever been any scientific analysis?")

ChatCompletion(id='chatcmpl-8hu9RCfMYFuRkEmqLGLfBS1QbRBOf', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Humor is subjective, and what one person finds funny, another may not. Therefore, it is challenging to determine the world\'s funniest joke objectively. However, there have been several attempts at analyzing humor scientifically. One notable example is the "LaughLab" experiment conducted by psychologist Richard Wiseman in 2001. He created a website where people could submit and rate jokes, ultimately gathering data from over 40,000 jokes. The results indicated that the following joke was considered the funniest:\n\n"Two hunters are out in the woods when one of them collapses. He doesn\'t seem to be breathing, and his eyes are glazed. The other guy whips out his phone and calls emergency services. He gasps, \'My friend is dead! What can I do?\' The operator says, \'Calm down, I can help. First, let\'s make sure he\'s dead.\' The